Here, we will get the festival happening in Switzerland, using data from:
[BandsInTown API](http://www.bandsintown.com/api/overview).

## 1. Scrapping
Data that will be received and stored:
- 'id'
- 'bandsintown url'
- 'datetime'
- 'artists'
    - per artist:
        - 'name'
        - 'bandsintown url'
- 'venue'
    - 'id'
    - 'name'
    - 'bandsintown url'
    - 'city'
    - 'region'
    - 'latitude'
    - 'longitude'

Limitations
* The API limits the number of returned results, so a request for every day will be done.
* The API limits the radius to 150 miles so several cities will be specified. To avoid overlap, the detail of the events will be checked and remove if they are the same
Cities : Geneva, Lausanne, Zurich, Berne, Basel, Lugano, Lucerne



In [1]:
import pandas as pd
import bandsInTownHelper as bandsInTownHelper

We will now extract the data. A csv file for each month will be created. 
Afterwards, all the csv will be aggregated in one big csv file and a dataframe.

In [2]:
starting_year =  2006
ending_year =  2017
    
bandsInTownHelper.getDataBandsInTown(starting_year,ending_year)


12 / 2017  saved to file


In [4]:
df = bandsInTownHelper.concatenateDataBandsInTown()

# 2. Data cleaning